# Bonus Task

This notebook contains the solution for the task #3, management of a Virtual Power Plant with stochasticity. <br> <br>
For reference:
<br>
Morales, J.M., Conejo, A.J., Madsen, H., Pinson, P. and Zugno, M., 2013. Integrating renewables in electricity
markets: operational problems (Vol. 205). Springer Science & Business Media., Chapter 8.

In [149]:
using JuMP, HiGHS, Plots, DataFrames

In [150]:
model = Model(HiGHS.Optimizer)
#set_silent(model)

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: HiGHS

## Compose Model

### Variables and Functions

In [151]:
T = ["t1", "t2", "t3"] # Time steps
Ω = ["ω1", "ω2", "ω3"] # Scenarios

I = [ "i1"] # power plants

Q = ["q1"] # wind generation units

J = ["j1"] # flexible loads (energy consumers)

K = ["k1"] # storage units (batteries etc.)

λ = Dict("t1" => 20, "t2" => 80, "t3" => 45) # Market price 
π = Dict("ω1" => 0.2, "ω2" => 0.3, "ω3" => 0.5) # probabilities of scenarios

# Cost
EG_coeff = Dict("i1" => 30) # Linear coefficient of power plant cost function’
EG_intersect = Dict("i1" => 50) #’No-load cost’

# Utility
EL_coeff = Dict("j1" => 90) # Linear coeffcient of flexible load utility function 
EL_intersect = Dict("j1" => 5) # Constant term of flexible load utility function 

# Min and max power output of power plant i
PG_min = Dict( "i1" => 1)
PG_max = Dict("i1" => 5)


# Min, max power consumption of flexible load j
PL_min = Dict( "j1" => 0.5)
PL_max = Dict("j1" => 2)
Min_Con = Dict("j1" => 2.5) # Minimum power consumption at the end of t3


# Min, max capacity of storage unit k
ES_max = Dict( "k1" => 1) 
ES_min = Dict( "k1" => 0.2) 
# Intital storage level
ES_0 = Dict( "k1" => 0.4) 
# (Dis)charging power limit
PS_charge_max = Dict( "k1" => 0.3)
PS_discharge_max = Dict( "k1" => 0.5)
PS_eff = Dict("k1" => 0.80) # Efficiency of (dis)charging storage unit k

# Wind generation values for the specific scenarios and time steps
PW_values = Dict(
    ("q1", "ω1", "t1") => 2.5, ("q1", "ω1", "t2") => 4.0, ("q1", "ω1", "t3") => 6.0,
    ("q1", "ω2", "t1") => 6.0, ("q1", "ω2", "t2") => 4.0, ("q1", "ω2", "t3") => 3.5,
    ("q1", "ω3", "t1") => 2.0, ("q1", "ω3", "t2") => 1.1, ("q1", "ω3", "t3") => 1.5
)

# Initialize an empty dictionary for PW
PW = Dict()

# Populate the PW dictionary with values from PW_values
for q in Q
    for ω in Ω
        for t in T
            PW[(q, ω, t)] = PW_values[(q, ω, t)]
        end
    end
end

In [152]:
# For time steps
function previous_time_period(t, T)
    index = findfirst(isequal(t), T)
    return T[index - 1]
end


previous_time_period (generic function with 1 method)

In [153]:
@variables(model, begin
        
    PD[T] # power exchanged with the electricity market at time step t
    EG[I,Ω,T] >= 0 # energy generated by power plant i in scenario ω and time step t
    C[I,Ω,T] >= 0  # Cost function for each i, ω, t
    U[J,Ω,T] >= 0 # Utility function for each j, ω, t
  #  PW[Q,Ω,T] >= 0 # energy generated by wind generation unit q in scenario ω and time step t ------------------------> depends on the scenario!!!
    PW_curt[Q,Ω,T] >= 0 # curtailment of energy generated by wind generation unit q in scenario ω and time step t
    PS_discharge[K,Ω,T] >= 0 # power discharged from storage unit k in scenario ω and time step t
    PS_charge[K,Ω,T] >= 0 # power charged to storage unit k in scenario ω and time step t
    ES[K,Ω,T] >= 0 # energy status of storage unit k in scenario ω and time step t
    EL[J,Ω,T] >= 0 # electricity consumed by flexible load j in scenario ω and time step t
    v[I,Ω,T], Bin # binary variable that declares whether or not power plant i is on/off at time step t
    
end)

(1-dimensional DenseAxisArray{VariableRef,1,...} with index sets:
    Dimension 1, ["t1", "t2", "t3"]
And data, a 3-element Vector{VariableRef}:
 PD[t1]
 PD[t2]
 PD[t3], 3-dimensional DenseAxisArray{VariableRef,3,...} with index sets:
    Dimension 1, ["i1"]
    Dimension 2, ["ω1", "ω2", "ω3"]
    Dimension 3, ["t1", "t2", "t3"]
And data, a 1×3×3 Array{VariableRef, 3}:
[:, :, "t1"] =
 EG[i1,ω1,t1]  EG[i1,ω2,t1]  EG[i1,ω3,t1]

[:, :, "t2"] =
 EG[i1,ω1,t2]  EG[i1,ω2,t2]  EG[i1,ω3,t2]

[:, :, "t3"] =
 EG[i1,ω1,t3]  EG[i1,ω2,t3]  EG[i1,ω3,t3], 3-dimensional DenseAxisArray{VariableRef,3,...} with index sets:
    Dimension 1, ["i1"]
    Dimension 2, ["ω1", "ω2", "ω3"]
    Dimension 3, ["t1", "t2", "t3"]
And data, a 1×3×3 Array{VariableRef, 3}:
[:, :, "t1"] =
 C[i1,ω1,t1]  C[i1,ω2,t1]  C[i1,ω3,t1]

[:, :, "t2"] =
 C[i1,ω1,t2]  C[i1,ω2,t2]  C[i1,ω3,t2]

[:, :, "t3"] =
 C[i1,ω1,t3]  C[i1,ω2,t3]  C[i1,ω3,t3], 3-dimensional DenseAxisArray{VariableRef,3,...} with index sets:
    Dimension 1, ["j1"

### Constraints

In [154]:
# Energy balance constraint
@constraint(model, E1[ω in Ω, t in T], 
    sum( EG[i,ω,t] for i in I) + (sum( PW[q,ω,t] - PW_curt[q,ω,t] for q in Q) + sum( PS_discharge[k,ω,t] for k in K))
    == sum( EL[j,ω,t] for j in J) + (sum( PS_charge[k,ω,t] for k in K) + PD[t])
)


2-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape},2,...} with index sets:
    Dimension 1, ["ω1", "ω2", "ω3"]
    Dimension 2, ["t1", "t2", "t3"]
And data, a 3×3 Matrix{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 E1[ω1,t1] : -PD[t1] + EG[i1,ω1,t1] - PW_curt[q1,ω1,t1] + PS_discharge[k1,ω1,t1] - PS_charge[k1,ω1,t1] - EL[j1,ω1,t1] == -2.5  …  E1[ω1,t3] : -PD[t3] + EG[i1,ω1,t3] - PW_curt[q1,ω1,t3] + PS_discharge[k1,ω1,t3] - PS_charge[k1,ω1,t3] - EL[j1,ω1,t3] == -6
 E1[ω2,t1] : -PD[t1] + EG[i1,ω2,t1] - PW_curt[q1,ω2,t1] + PS_discharge[k1,ω2,t1] - PS_charge[k1,ω2,t1] - EL[j1,ω2,t1] == -6       E1[ω2,t3] : -PD[t3] + EG[i1,ω2,t3] - PW_curt[q1,ω2,t3] + PS_discharge[k1,ω2,t3] - PS_charge[k1,ω2,t3] - EL[j1,ω2,t3] == -3.5
 E1[ω3,t1] : -PD[t1] + EG[i1,ω3,t1] - PW

In [155]:
@constraints(model, begin
    # Energy generation constraint of power plants 
    E2[i in I, ω in Ω, t in T], v[i,ω,t] * PG_min[i] <= EG[i,ω,t] 
    E3[i in I, ω in Ω, t in T], EG[i,ω,t] <= v[i,ω,t] * PG_max[i]  

    # Energy demand constraints of flexible loads  
    E4[j in J, ω in Ω, t in T], PL_min[j] <= EL[j,ω,t]  
    E5[j in J, ω in Ω, t in T], EL[j,ω,t] <= PL_max[j]
    E6[j in J, ω in Ω], sum(EL[j,ω,t] for t in T) >= Min_Con[j]

    # Energy level of the battery

    # Consider battery SOC when starting
    E7[k in K, ω in Ω], ES[k,ω,"t1"] == ES_0[k] + (PS_eff[k] * PS_charge[k,ω,"t1"]) - ((1/PS_eff[k]) * PS_discharge[k,ω,"t1"])
    E8[k in K, ω in Ω, t in T[2:end]], ES[k, ω, t] == ES[k, ω, previous_time_period(t, T)] + (PS_eff[k] * PS_charge[k, ω, t]) - ((1/PS_eff[k]) * PS_discharge[k, ω, t])  
    # Max and min SOC
    E9[k in K, ω in Ω, t in T], ES_min[k] <= ES[k, ω, t]
    E10[k in K, ω in Ω, t in T], ES[k, ω, t] <= ES_max[k] 

    # Max and min (dis)charging rates

    E11[k in K, ω in Ω, t in T], 0 <= PS_charge[k, ω, t]
    E12[k in K, ω in Ω, t in T], PS_charge[k, ω, t] <= PS_charge_max[k]  
    E13[k in K, ω in Ω, t in T], 0 <= PS_discharge[k, ω, t]
    E14[k in K, ω in Ω, t in T], PS_discharge[k, ω, t] <= PS_discharge_max[k]
        
    # Power generated by wind power unit in different scenarios and time steps
    cost_function[i in I, ω in Ω, t in T], C[i, ω, t] == EG_coeff[i] * EG[i, ω, t] + EG_intersect[i]
    utility_function[j in J, ω in Ω, t in T], U[j, ω, t] == EL_coeff[j] * (EL[j, ω, t]) + EL_intersect[j]
        
    # Maximum wind curtailment of wind power unit
    E24[q in Q, ω in Ω, t in T], PW_curt[q,ω,t] >= 0
    E25[q in Q, ω in Ω, t in T], PW_curt[q,ω,t] <= PW[q,ω,t] 

end)

(3-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape},3,...} with index sets:
    Dimension 1, ["i1"]
    Dimension 2, ["ω1", "ω2", "ω3"]
    Dimension 3, ["t1", "t2", "t3"]
And data, a 1×3×3 Array{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}, 3}:
[:, :, "t1"] =
 E2[i1,ω1,t1] : -EG[i1,ω1,t1] + v[i1,ω1,t1] <= 0  …  E2[i1,ω3,t1] : -EG[i1,ω3,t1] + v[i1,ω3,t1] <= 0

[:, :, "t2"] =
 E2[i1,ω1,t2] : -EG[i1,ω1,t2] + v[i1,ω1,t2] <= 0  …  E2[i1,ω3,t2] : -EG[i1,ω3,t2] + v[i1,ω3,t2] <= 0

[:, :, "t3"] =
 E2[i1,ω1,t3] : -EG[i1,ω1,t3] + v[i1,ω1,t3] <= 0  …  E2[i1,ω3,t3] : -EG[i1,ω3,t3] + v[i1,ω3,t3] <= 0, 3-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}

### Optimization Objective

In [156]:
@objective(model, Max,
    sum(
    ( λ[t] * PD[t]) + sum(π[ω] * ( sum( U[j,ω,t] for j in J) - ( sum( C[i,ω,t] for i in I))) for ω in Ω) for t in T))


20 PD[t1] + 0.2 U[j1,ω1,t1] - 0.2 C[i1,ω1,t1] + 0.3 U[j1,ω2,t1] - 0.3 C[i1,ω2,t1] + 0.5 U[j1,ω3,t1] - 0.5 C[i1,ω3,t1] + 80 PD[t2] + 0.2 U[j1,ω1,t2] - 0.2 C[i1,ω1,t2] + 0.3 U[j1,ω2,t2] - 0.3 C[i1,ω2,t2] + 0.5 U[j1,ω3,t2] - 0.5 C[i1,ω3,t2] + 45 PD[t3] + 0.2 U[j1,ω1,t3] - 0.2 C[i1,ω1,t3] + 0.3 U[j1,ω2,t3] - 0.3 C[i1,ω2,t3] + 0.5 U[j1,ω3,t3] - 0.5 C[i1,ω3,t3]

In [157]:
print(model)

In [158]:
optimize!(model)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
39 rows, 66 cols, 132 nonzeros
30 rows, 57 cols, 150 nonzeros
30 rows, 57 cols, 150 nonzeros

Solving MIP model with:
   30 rows
   57 cols (9 binary, 0 integer, 0 implied int., 48 continuous)
   150 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   956.56          -inf                 inf        0      0      0         0     0.0s
 R       0       0         0   0.00%   779.008         611.66            27.36%        0      0      0        20     0.0s

11.1% inactive integer columns, restarting
Model after restart has 29 rows, 56 cols (8 bin., 0 int., 0 impl., 48 cont.), and 143 nonzeros

         0       0         0   0.00%   779.008         611.66            2

In [159]:
termination_status(model)

OPTIMAL::TerminationStatusCode = 1

In [160]:
objective_value(model)

777.1605

In [161]:
value.(PW_curt)

3-dimensional DenseAxisArray{Float64,3,...} with index sets:
    Dimension 1, ["q1"]
    Dimension 2, ["ω1", "ω2", "ω3"]
    Dimension 3, ["t1", "t2", "t3"]
And data, a 1×3×3 Array{Float64, 3}:
[:, :, "t1"] =
 0.0  2.5  0.0

[:, :, "t2"] =
 0.0  0.0  0.0

[:, :, "t3"] =
 0.0  0.0  0.0

In [162]:
value.(PD)

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, ["t1", "t2", "t3"]
And data, a 3-element Vector{Float64}:
 1.2000000000000008
 5.952
 4.5

In [163]:
value.(ES)

3-dimensional DenseAxisArray{Float64,3,...} with index sets:
    Dimension 1, ["k1"]
    Dimension 2, ["ω1", "ω2", "ω3"]
    Dimension 3, ["t1", "t2", "t3"]
And data, a 1×3×3 Array{Float64, 3}:
[:, :, "t1"] =
 0.64  0.64  0.64

[:, :, "t2"] =
 0.825  0.2  0.2

[:, :, "t3"] =
 0.2  0.2  0.2

In [164]:
value.(EL)

3-dimensional DenseAxisArray{Float64,3,...} with index sets:
    Dimension 1, ["j1"]
    Dimension 2, ["ω1", "ω2", "ω3"]
    Dimension 3, ["t1", "t2", "t3"]
And data, a 1×3×3 Array{Float64, 3}:
[:, :, "t1"] =
 2.0  2.0  2.0

[:, :, "t2"] =
 2.0  2.0  0.5

[:, :, "t3"] =
 2.0  2.0  2.0